In [9]:
import numpy as np
from numpy import genfromtxt

from scipy import *
from scipy.linalg import norm, pinv, det
from scipy.optimize import minimize
import csv
import cvxopt as opt
from cvxopt import matrix, solvers

import matplotlib.pyplot as plt

from PIL import Image
import matplotlib.image as mping

import pandas as pd
from __future__ import division


In [10]:
#Read the data 
Xtr = genfromtxt('Xtr.csv', delimiter = ',')
d = Xtr.shape[1]

Xtr = np.delete(Xtr, (d-1), axis = 1)

Ytr = pd.read_csv('Ytr.csv')
Ytr = np.array(Ytr)
Ytr = np.delete(Ytr, (0), axis = 1)

#Xtr is a 5000 * 3072 dimensional matrix

#Ytr is a 5000*1 dimensional matrix



In [11]:
#This function crops the image and detects the shape of the target object 

def Crop(imagecol):
    imagecol = np.reshape(imagecol,(3,32,32))
    image_bw = (0.21*imagecol[0,:,:]+0.72*imagecol[1,:,:]+ 0.07*imagecol[2,:,:])
    first_channel_kept=[]
    second_channel_kept=[]
    third_channel_kept=[]
    
    m=mean(image_bw)
    diffpos=(image_bw.max()-m)* 0.05
    diffneg=(m-image_bw.min())* 0.05
    threspos=m+diffpos
    thresneg=m-diffneg
    for line in range(image_bw.shape[0]):
        
        select_indices = np.argwhere((image_bw[line,:] > threspos) | (image_bw[line,:]<thresneg ))
        #print(len(select_indices))
        if(len(select_indices)>1):
            firstIndex=select_indices[0][0]
            secondIndex=min(select_indices[-1][0]+1,32)
            first_channel_kept=np.concatenate([first_channel_kept,imagecol[0,line,firstIndex:secondIndex]])
            second_channel_kept=np.concatenate([second_channel_kept,imagecol[1,line,firstIndex:secondIndex]])
            third_channel_kept=np.concatenate([third_channel_kept,imagecol[2,line,firstIndex:secondIndex]])
   
    
    cropped_image_col=np.zeros((3,first_channel_kept.shape[0]))
    #[h,s,v]=rgb_to_hsv(first_channel_kept,second_channel_kept,third_channel_kept)
    cropped_image_col[0]=first_channel_kept
    cropped_image_col[1]=second_channel_kept
    cropped_image_col[2]=third_channel_kept
    
    return cropped_image_col

In [12]:
#The block to display images

def displayImage(image):
    plt.imshow(image,cmap='Greys')
    plt.show()

In [13]:
#Preprocessing images
#Code to tranform images into 16*16*16 dimensional histograms

Xtilde = np.reshape(Xtr, (5000, 3, 1024))

minR = np.min(Xtilde[:,0,:])
minG = np.min(Xtilde[:,1,:])
minB = np.min(Xtilde[:,2,:])

maxR = np.max(Xtilde[:,0,:])
maxG = np.max(Xtilde[:,1,:])
maxB = np.max(Xtilde[:,2,:])

def Compute_Histogram(Im, nbBins):
    
    Image = Im
    R = Image[0,:]
    G = Image[1,:]
    B = Image[2,:]
    
    LR = np.linspace(minR, maxR, nbBins)
    LG = np.linspace(minG, maxG, nbBins)
    LB = np.linspace(minB, maxB, nbBins)
    
    
    h = np.zeros((nbBins,nbBins,nbBins))
    
    nbpixel = 0
    for i in range(nbBins-1):
        for j in range(nbBins-1):
            for k in range(nbBins-1):
                h[i,j,k] = (np.where((R>=LR[i]) & (R < LR[i+1]) & (G>=LG[j]) & (G < LG[j+1]) &(B>=LB[k]) & (B < LB[k+1]))[0]).shape[0]
                nbpixel += h[i,j,k] 
    h = h/nbpixel
    h = np.reshape(h, nbBins**3)
    return h


In [15]:
#In this block we fill the new images (encoded with 16 bins) and the histograms
nbBins = 16

Histogram = np.zeros((Xtr.shape[0], nbBins**3))
#Histogram is a 5000 * 16^3 dimensional matrix


for i in range(Xtr.shape[0]):
    Im = Xtr[i]
    Im = np.reshape(Im,(3,1024))
    Histogram[i,:] = Compute_Histogram(Im, nbBins)
    


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [16]:
#Compute a distance for histograms
def distance(x,y):
    index = np.where((x != 0) | (y != 0))[0]
    d = np.sum((x[index]-y[index])**2/(x[index]+y[index]))
    return d


def Quadratic_Kernel(x,y):
    return  (np.dot(x,y)+1)**2

    
    
def KernelComputation(x):
    K = np.zeros((x.shape[0],x.shape[0]))
    for line in range(x.shape[0]):
        print line
        for col in range(x.shape[0]):
            if (line<=col):
                value= Quadratic_Kernel(x[line,:],x[col,:])
                K[col,line]=value
                K[line,col]=value
    return K

#Here we the matrix K
K = KernelComputation(Histogram)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [17]:
#I mmodified a bit th Quadratic_Kernel to test the efficiency of a polynomial kernel. 
#Clearly we should better use a polynomial kernel with a degree greater or equal than 2


#Return x (images) and y corresponding to label1 and label2 
def Fill_x_y_label(X, label1,label2):

    x1=X[np.where(Ytr==label1)[0],:]
    x2=X[np.where(Ytr==label2)[0],:]
    x=np.concatenate([x1,x2])
    
    y1=np.ones(x1.shape[0])
    y2=-1*np.ones(x2.shape[0])
    y=np.concatenate([y1,y2])
    return x,y;

In [26]:
#Cross-validation procedure
NbTrain =400
#NbTrain is a number of training data labeled by 1 I extract from the initial dataset
#NbTrain =400 in our case. I consider 400 data points of label 1 in tmy training set

def Fill_Training_Test_Set(X, label1, label2, Kernel):
    NbTest = 500-NbTrain
    
    x, y = Fill_x_y_label(X,label1,label2)
    
    yTrain = np.concatenate([np.ones(NbTrain), -np.ones(NbTrain)])
    xTrain = np.concatenate([x[0:NbTrain,:],x[500: 500 + NbTrain,:]])
    yTest = np.concatenate([np.ones(NbTest),-np.ones(NbTest)])
    xTest = np.concatenate([x[NbTrain:500,:],x[500 + NbTrain: 1000,:]])
    
    index  = np.concatenate([np.where(Ytr==label1)[0][0:NbTrain], np.where(Ytr==label2)[0][0:NbTrain]])
    KTrain= Kernel[index,:]
    KTrain = KTrain[:,index]
    return xTrain,yTrain, xTest, yTest,KTrain



In [27]:
#Optimization model
def Training_model(y, Kernel, Lambda):
    P = matrix(2*Kernel)
    q = matrix(-2*y)
    G = matrix(np.concatenate(((-1)*np.diag(y),np.diag(y))), tc='d')
    h = matrix(np.concatenate([np.zeros(y.shape), (1/(2*float(Lambda*y.shape[0])))*np.ones(y.shape)]), tc='d')
    A = matrix(np.ones(y.shape[0]),(1,y.shape[0]))
    b = matrix(0.0)
    sol = solvers.qp(P,q,G,h,A,b)
    alpha = sol['x']
    print sol['status']
    alpha = np.array(alpha)
    return alpha


#Compute the new obtained label
def SVM_function(alpha, xtrain ,x_querry):
    value = 0
    for i in range(alpha.shape[0]):
        Current_Image=xtrain[i,:]
        value += alpha[i]*Quadratic_Kernel(x_querry,Current_Image)
    return value


In [30]:
#Block to perform the optimization with the Quadratic_Kernel

#Just a test for the one vs one classifier 
Lambda=0.00001
NbLabel =10
NbTest = 2*(500-NbTrain)



AlphaClassifiers = np.zeros((NbLabel,NbLabel,N))
b = np.zeros((NbLabel,NbLabel))

#The following block tests all duels between labels
for label1 in range(NbLabel):
    for label2 in range(NbLabel):
        if(label2 > label1):
            xTrain, yTrain, xTest, yTest ,KTrain = Fill_Training_Test_Set(Histogram, label1, label2, K)
            alpha  = Training_model(yTrain, KTrain, Lambda)
            count =0
            for i in range(alpha.shape[0]):
                if((alpha[i]*yTrain[i] > 0) and (alpha[i]*yTrain[i] < (1/(2*Lambda*yTrain.shape[0])))):
                    count +=1
                    b[label1,label2] += yTrain[i]
                    for j in range(alpha.shape[0]):
                        b[label1,label2] -= alpha[j]*Quadratic_Kernel(xTrain[i,:],xTrain[j,:])
            b[label1,label2]=b[label1,label2]/count
            AlphaClassifiers[label1, label2, :] = alpha[:,0]
            error = 0
            for i in range(NbTest):
                Label_found = sign(SVM_function(alpha, xTrain, xTest[i,:])+ b[label1, label2])
                if(Label_found != yTest[i]):
                    error += 1
            Accuracy = (1-(error)/NbTest)*100.0
            print 'Accuracy :', Accuracy, '%'

     pcost       dcost       gap    pres   dres
 0: -3.8212e+04 -7.4388e+05  7e+05  6e-18  6e-12
 1: -5.0840e+04 -1.3218e+05  8e+04  2e-16  8e-12
 2: -7.4236e+04 -9.0631e+04  2e+04  2e-16  1e-11
 3: -8.1043e+04 -8.4441e+04  3e+03  2e-16  1e-11
 4: -8.2120e+04 -8.3461e+04  1e+03  2e-16  1e-11
 5: -8.2635e+04 -8.2921e+04  3e+02  2e-16  1e-11
 6: -8.2752e+04 -8.2799e+04  5e+01  2e-16  1e-11
 7: -8.2773e+04 -8.2778e+04  5e+00  2e-16  1e-11
 8: -8.2775e+04 -8.2776e+04  3e-01  2e-16  1e-11
 9: -8.2775e+04 -8.2775e+04  4e-03  2e-16  1e-11
Optimal solution found.
optimal
Accuracy : 75.0 %
     pcost       dcost       gap    pres   dres
 0: -1.1617e+04 -1.9183e+06  4e+06  6e-01  4e-12
 1: -7.0999e+03 -5.3235e+05  5e+05  2e-16  4e-12
 2: -2.3816e+04 -1.1310e+05  9e+04  2e-16  4e-12
 3: -3.0751e+04 -6.6448e+04  4e+04  2e-16  4e-12
 4: -3.3844e+04 -5.8775e+04  2e+04  2e-16  4e-12
 5: -3.7500e+04 -4.8100e+04  1e+04  2e-16  5e-12
 6: -3.9215e+04 -4.4396e+04  5e+03  2e-16  5e-12
 7: -4.0094e+04 -4.26

In [29]:
def ToTal_set(X):
    xtest =np.zeros((1,X.shape[1]))
    ytest = np.zeros((1,1))
    xtrain = np.zeros((NbLabel,NbTrain,X.shape[1]))
    for label in range(NbLabel):
        x1=X[np.where(Ytr==label)[0],:]
        xtrain[label,:,:] = x1[0: NbTrain,:]
        xtest=np.concatenate([xtest,x1[NbTrain: 500,:]])
        y1 = label * np.ones((500 - NbTrain, 1))
        ytest=np.concatenate([ytest,y1])
    xtest = np.delete(xtest, 0, 0)
    ytest = np.delete(ytest, 0, 0)
    return xtrain, xtest, ytest

threshold = 0.7
    
def FindLabels(xtrain, Image):
    delta = np.zeros((NbLabel,1))
    for label1 in range(NbLabel):
        for label2 in range(NbLabel):
            if(label2 > label1):
                current_training_set = np.concatenate([xtrain[label1,:,:],xtrain[label2,:,:]])
                value = SVM_function(AlphaClassifiers[label1,label2,:], current_training_set, Image) + b[label1,label2]
                
                if value > 0:
                    delta[label1,0] += 1
                if value < 0:
                    delta[label2,0] += 1
                    
    firstlabel = np.argmax(delta[:,0])
    bestClass = np.argwhere(delta[:,0] >= threshold * (NbLabel-1))
    bestClass = bestClass[:,0]
    count = 0
    while bestClass.shape[0] > 1:
        delta = np.zeros((NbLabel,1))
        for i in bestClass:
            for j in bestClass:
                if(i < j):
                    count +=1
                    current_training_set = np.concatenate([xtrain[i,:,:],xtrain[j,:,:]])
                    value = SVM_function(AlphaClassifiers[i,j,:], current_training_set, Image) + b[i,j]
                    if value > 0:
                        delta[i,0] += 1
                    else:
                        delta[j,0] += 1
        
        bestClass = np.argwhere(delta[:,0] >= threshold * (bestClass.shape[0]-1))
        bestClass = bestClass[:,0]
        print bestClass.shape
    if(bestClass.shape[0] == 0):
        labelFound = firstlabel
    else:
        labelFound = bestClass[0]
    return labelFound

NbTest = NbLabel*(500-NbTrain)

xtrain, xtest, ytest = ToTal_set(Histogram)
print xtest.shape

error =0


for i in range(NbTest):
    label_found = FindLabels(xtrain, xtest[i,:])
    print label_found
    if(label_found != ytest[i]):
        error = error +1
    Accuracy = (1-(float(error)/(i+1)))*100.0
    print 'Accuracy :', Accuracy, '%'

                
            

(20L, 4096L)
(1L,)
6
Accuracy : 0.0 %
(1L,)
8
Accuracy : 0.0 %
(1L,)
1
Accuracy : 33.3333333333 %
(1L,)
9
Accuracy : 25.0 %
(1L,)
8
Accuracy : 20.0 %
(1L,)
2
Accuracy : 33.3333333333 %
(1L,)
1
Accuracy : 28.5714285714 %
(1L,)
9
Accuracy : 25.0 %
(1L,)
8
Accuracy : 22.2222222222 %
(1L,)
3
Accuracy : 20.0 %
(1L,)
3
Accuracy : 18.1818181818 %
(0L,)
4
Accuracy : 16.6666666667 %
(1L,)
3
Accuracy : 15.3846153846 %
(1L,)
6
Accuracy : 21.4285714286 %
(1L,)
4
Accuracy : 20.0 %
(1L,)
7
Accuracy : 25.0 %
(0L,)
0
Accuracy : 23.5294117647 %
(1L,)
4
Accuracy : 22.2222222222 %
(1L,)
9
Accuracy : 26.3157894737 %
(1L,)
1
Accuracy : 25.0 %


In [31]:
Xte = genfromtxt('Xte.csv', delimiter = ',')
d = Xte.shape[1]

print Xte.shape
print Xte[1,:]

#Xte = np.delete(Xtr, (d-1), axis = 1)

(2000L, 3073L)
[  3.88091600e-02   1.74643800e-02   3.39512300e-02 ...,  -6.04765600e-04
   3.00709000e-05              nan]


In [32]:
Xte = np.delete(Xte, (d-1), axis = 1)
print Xte.shape

(2000L, 3072L)


In [34]:
nbBins = 16
HistogramTest = np.zeros((Xte.shape[0],nbBins**3))
for i in range(Xte.shape[0]):
    print i
    Im = Xte[i,:]
    Im = np.reshape(Im,(3,1024))
    #I = center_image(Im)
    #I = np.reshape(I, (3, 32, 32))
    #I = colors.rgb_to_hsv(I.T)
    #I = I.T
    #I= np.reshape(I, (3,1024))
    #I = (0.31*Im[0,:]+0.59*Im[1,:]+ 0.10*Im[2,:])
    #X[i,:], Histogram[i,:] = quantize(Im, nbBins)
    
    HistogramTest[i,:] = Compute_Histogram(Im, nbBins)
    
Yte = np.zeros(Xte.shape[0])
    
for i in range(Xte.shape[0]):
    print i
    label_found = FindLabels(xtrain, HistogramTest[i,:])
    Yte[i] = label_found

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [35]:
#This block creates the file to be submitted

with open('test.csv', 'wb') as f:
    #writer = csv.writer(output, lineterminator='\n')
    writer = csv.writer(f)
    writer.writerow(['Id','Prediction'])
    i=0
    for val in Yte:
        #newVal = int(val)
        
        writer.writerow([i+1,int(val)])
        i = i+1